In [1]:
import tensorflow as tf 
import keras 
import numpy as np 
import pandas as pd

In [2]:
def make_random():
    return [np.random.randint(1,10) for i in range(4)]

X = [make_random() for i in range(2000)]
Y=[]
counter = 3
temp = [] 
for i in range(len(X)):
    for j in range(len(X[i])) :
        temp.append(X[i][counter])
        counter -=1
    counter = 3
    Y.append(temp)
    temp = []

In [3]:
XS = keras.utils.to_categorical(X,num_classes=10)
YS = keras.utils.to_categorical(Y,num_classes=10)

In [4]:
YS[0]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
encoder_inputs = keras.Input(shape=(4,10))
encoder_lstm1 = keras.layers.LSTM(16,return_state=True)
encoder_output,state_h,state_c=encoder_lstm1(encoder_inputs)
states = [state_h,state_c]

In [6]:
print(state_c.shape)
print(state_h.shape)
print(encoder_output.shape)

(None, 16)
(None, 16)
(None, 16)


In [7]:
decoder_inputs = keras.Input(shape=(1,10))
decoder_lstm1 = keras.layers.LSTM(16,return_sequences=True,return_state=True)
decoder_outputs,state_C,state_H = decoder_lstm1(decoder_inputs,initial_state=states)
decoder_dense = keras.layers.Dense(10,activation='softmax')

In [20]:
def created_decoder_input_Models (batch_size) :
    #Making Encoder First 
    encoder_Input = keras.Input(shape=(4,10))
    encoder_lstm = keras.layers.LSTM(16,return_state=True)
    outputs_lstm,state_h,state_c = encoder_lstm(encoder_Input)

    #Initial state for decoder initial_state 
    states = [state_h,state_c]

    #Making Decoder Layers 
    #shape input will folowwing the state encoder shape 
    decoder_input = keras.Input(shape=(4,10))
    decoder_lstm = keras.layers.LSTM(16,return_state=True,return_sequences=True)
    decoder_dense = keras.layers.Dense(10,activation='softmax')

    all_outputs = []
    decoder_input_data = np.zeros((batch_size,1,10))
    decoder_input_data[:,0,0] = 1
    inputs = decoder_input_data
    for i in range(4) :
        outputs,state_h,state_c = decoder_lstm(inputs,initial_state=states)
        outputs = decoder_dense(outputs)
        all_outputs.append(outputs)
        print(f"Step {i+1}, outputs shape: {outputs.shape}")
        print(f"Step{i+1} shape of state h : {state_h.shape}" )
        print(f"Step{i+1} shape of state c : {state_c.shape}")
        inputs = outputs
        states = [state_h,state_c]
    decoder_outputs = keras.layers.Lambda(lambda x : keras.layers.concatenate(x,axis=1))(all_outputs)
    model = keras.Model(encoder_Input,decoder_outputs)
    for idx, output in enumerate(all_outputs):
        print(f"Output at timestep {idx+1} shape: {output.shape}")
        print("Final concatenated output shape:", decoder_outputs.shape)

    model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=['accuracy'])
    return model
    

In [25]:
encoder_decoder_models = created_decoder_input_Models(1)


Step 1, outputs shape: (1, 1, 10)
Step1 shape of state h : (1, 16)
Step1 shape of state c : (1, 16)
Step 2, outputs shape: (1, 1, 10)
Step2 shape of state h : (1, 16)
Step2 shape of state c : (1, 16)
Step 3, outputs shape: (1, 1, 10)
Step3 shape of state h : (1, 16)
Step3 shape of state c : (1, 16)
Step 4, outputs shape: (1, 1, 10)
Step4 shape of state h : (1, 16)
Step4 shape of state c : (1, 16)
Output at timestep 1 shape: (1, 1, 10)
Final concatenated output shape: (1, 4, 10)
Output at timestep 2 shape: (1, 1, 10)
Final concatenated output shape: (1, 4, 10)
Output at timestep 3 shape: (1, 1, 10)
Final concatenated output shape: (1, 4, 10)
Output at timestep 4 shape: (1, 1, 10)
Final concatenated output shape: (1, 4, 10)


In [10]:
XS[10]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [26]:
encoder_decoder_models.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 4, 10)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_12 (LSTM)      │ [(None, 16),      │      1,728 │ input_layer_12[0… │
│                     │ (None, 16),       │            │                   │
│                     │ (None, 16)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_13 (LSTM)      │ [(1, 1, 16), (1,  │      1,728 │ lstm_12[0][1],    │
│                     │ 16), (1, 16)]     │            │ lstm_12[0][2],    │
│                     │                   │            │ dense_6[0][0],    │
│                     │                   │            │ lstm_13[0][1],    │
│                     │                   │            │ lstm_13[0][2],    │
│                     │                   │            │ dense_6[1][0],    │
│                     │                   │            │ lstm_13[1][1],    │
│                     │                   │            │ lstm_13[1][2],    │
│                     │                   │            │ dense_6[2][0],    │
│                     │                   │            │ lstm_13[2][1],    │
│                     │                   │            │ lstm_13[2][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (1, 1, 10)        │        170 │ lstm_13[0][0],    │
│                     │                   │            │ lstm_13[1][0],    │
│                     │                   │            │ lstm_13[2][0],    │
│                     │                   │            │ lstm_13[3][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (1, 4, 10)        │          0 │ dense_6[0][0],    │
│                     │                   │            │ dense_6[1][0],    │
│                     │                   │            │ dense_6[2][0],    │
│                     │                   │            │ dense_6[3][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,626 (14.16 KB)

 Trainable params: 3,626 (14.16 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
encoder_decoder_models.fit(XS,YS,verbose=1,epochs=40)

Epoch 1/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.1020 - loss: 2.2859
Epoch 2/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2002 - loss: 2.1951
Epoch 3/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.2901 - loss: 2.0836
Epoch 4/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3257 - loss: 1.9147
Epoch 5/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3635 - loss: 1.7496
Epoch 6/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3823 - loss: 1.6450
Epoch 7/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3873 - loss: 1.5795
Epoch 8/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4056 - loss: 1.5154
Epoch 9/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4045 - loss: 1.4904
Epoch 10/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4090 - loss: 1.4389
Epoch 11/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4161 - loss: 1.4079
Epoch 12/40
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.42

In [53]:
data = np.array([XS[0],XS[1],XS[2],XS[3],XS[4],XS[5],XS[6]])

In [54]:
data

array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0

In [55]:
predicted = encoder_decoder_models.predict(data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [56]:
result = []
result2=[]
for i in range(len(data)):
    for j in range(len(data[i])) :
        result.append(np.argmax(data[i][j]))
    result2.append(result)
    result=[]

In [57]:
result2

[[2, 1, 1, 4],
 [9, 1, 3, 6],
 [7, 8, 7, 7],
 [5, 2, 1, 3],
 [1, 8, 2, 9],
 [9, 5, 5, 9],
 [3, 2, 1, 3]]

In [59]:
Y[6]

[3, 1, 2, 3]